In [1]:
import numpy as np
from typing import Tuple, List, Optional
from scipy.spatial.transform import Rotation as R

In [2]:
file_path = "data/test.bvh"

In [3]:
joint_name = []
joint_parent = []
joint_offset = np.empty((10, 3))

In [16]:
f = open(file_path, 'r')

In [18]:
f.tell(), f.readline(), f.tell()

(11, 'ROOT RootJoint\n', 27)

In [23]:
f.seek(12)

12

In [24]:
f.readline()

'OOT RootJoint\n'

In [13]:
type(f)

_io.TextIOWrapper

In [16]:
with open(file_path, 'r') as f:  
    lines = f.readlines()  
    print(lines)

['HIERARCHY\n', 'ROOT RootJoint\n', '{\n', '    OFFSET   0.000000   0.000000   0.000000\n', '    CHANNELS 6 Xposition Yposition Zposition Xrotation Yrotation Zrotation\n', '    JOINT lHip\n', '    {\n', '        OFFSET   0.100000  -0.051395   0.000000\n', '        CHANNELS 3 Xrotation Yrotation Zrotation\n', '        End Site\n', '        {\n', '            OFFSET   0.010000   0.002000   0.060000\n', '        }\n', '    }\n', '    JOINT rHip\n', '    {\n', '        OFFSET  -0.100000  -0.051395   0.000000\n', '        CHANNELS 3 Xrotation Yrotation Zrotation\n', '        JOINT rKnee\n', '        {\n', '            OFFSET   0.000000  -0.410000   0.000000\n', '            CHANNELS 3 Xrotation Yrotation Zrotation\n', '            End Site\n', '            {\n', '                OFFSET  -0.010000   0.002000   0.060000\n', '            }\n', '        }\n', '    }\n', '}\n', 'MOTION\n', 'Frames: 182\n', 'Frame Time:   0.016667\n', ' -0.001735   0.855388   0.315499   2.008551   7.606260  -0.79

In [29]:
index = 0
lines[index], lines[index].strip()

('HIERARCHY\n', 'HIERARCHY')

In [32]:
# startwith
# split
# strip

True

In [3]:
# import numpy as np
# 
# def parse_bvh_file(bvh_file):
#     joint_names = []
#     parent_indices = []
#     offsets = []
# 
#     with open(bvh_file, 'r') as file:
#         lines = file.readlines()
# 
#     def parse_joint(lines, index, parent_name):
#         while index < len(lines):
#             line = lines[index].strip()
#             if line.startswith("JOINT") or line.startswith("End Site"):
#                 joint_name = line.split()[-1] if line.startswith("JOINT") else parent_name + "_end"
#                 joint_names.append(joint_name)
#                 parent_indices.append(joint_names.index(parent_name) if parent_name else -1)
# 
#                 offset_line = lines[index + 1].strip()
#                 if offset_line.startswith("OFFSET"):
#                     offset = np.array([float(val) for val in offset_line.split()[1:]])
#                     offsets.append(offset)
# 
#                 if line.startswith("JOINT"):
#                     index, _ = parse_joint(lines, index + 2, joint_name)
#                 elif line.startswith("End Site"):
#                     index += 2
#             elif line.startswith("}"):
#                 break
#             index += 1
# 
#         return index, joint_names
# 
#     parse_joint(lines, 0, None)
# 
#     return joint_names, parent_indices, np.array(offsets)
# 
# bvh_file = "data/test.bvh"
# joint_names, parent_indices, offsets = parse_bvh_file(bvh_file)
# 
# # Print the results
# for i in range(len(joint_names)):
#     parent_name = joint_names[parent_indices[i]] if parent_indices[i] != -1 else "ROOT"
#     print(f"Joint: {joint_names[i]}, Parent: {parent_name}, Offset: {offsets[i]}")


IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
# joint_names, parent_indices, offsets

(['lHip', 'lHip_end'], [-1, 0], array([], dtype=float64))

In [ ]:
def bvh_end_of_branch(lines: List[str],
                      i: int) -> int:
    while not lines[i].strip().startswith("JOINT"):
        i += 1
        
    return i - 1

        
def pose_from_bvh_lines(lines: List[str], 
                        joint_names: List[str],
                        joint_parents: List[int],
                        joint_offsets: List[List[float]],
                        i: int = 0,
                        parent_index: int = -1) -> Optional[int]:
    my_joint_name_read = False
    my_name_index = 0

    while lines[i] != "MOTION\n":
        line = lines[i].strip()

        if line.startswith("JOINT") or line.startswith("ROOT"):
            if my_joint_name_read:
                i = pose_from_bvh_lines(lines,
                                        joint_names, 
                                        joint_parents,
                                        joint_offsets,
                                        i,
                                        my_name_index)
            else:
                joint_parents.append(parent_index)

                my_name_index = len(joint_names)
                joint_names.append(line.split()[-1])
                my_joint_name_read = True
                
        elif line.startswith("OFFSET"):
            joint_offsets.append([float(offset) for offset in line.split()[-3:]])
        
        elif line.startswith("End"):
            joint_parents.append(parent_index)
            
            joint_names.append(joint_names[parent_index] + "_end")
            joint_offsets.append([float(offset) for offset in lines[i + 2].split()[-3:]])
            return bvh_end_of_branch(lines, i)
        
        i += 1
        
    return 
    

def part1_calculate_T_pose(bvh_file_path: str) -> Tuple[List[str], List[int], np.ndarray]:
    
    joint_names = []
    joint_parents = []
    joint_offsets = []
    
    with open(bvh_file_path, 'r') as bvh:
        lines = bvh.readlines()
        
        pose_from_bvh_lines(lines,
                            joint_names, 
                            joint_parents, 
                            joint_offsets)
            
    return joint_names, joint_parents, np.array(joint_offsets, dtype=np.float64)
    

In [ ]:
def pose_from_bvh_file(file, 
                        joint_names: List[str],
                        joint_parents: List[int],
                        joint_offsets: List[List[float]],
                        parent_index: int = -1) -> None:
    my_joint_name_read = False
    my_name_index = 0

    line = file.nextline().strip()
    while line != "MOTION":
            
        if line.startswith("JOINT") or line.startswith("ROOT"):
            if my_joint_name_read:
                file.seek(-1, 1)
                pose_from_bvh_file(file,
                                    joint_names, 
                                    joint_parents,
                                    joint_offsets,
                                    my_name_index)
            else:
                joint_parents.append(parent_index)
                
                my_name_index = len(joint_names)
                joint_names.append(line.split()[-1])
                my_joint_name_read = True
                
        elif line.startswith("OFFSET"):
            joint_offsets.append([float(offset) for offset in line.split()[-3:]])
        
        elif line.startswith("End"):
            joint_parents.append(parent_index)            

            joint_names.append(joint_names[parent_index] + "_end")
            file.seek(1, 1) # down by 1 or 2?
            joint_offsets.append([float(offset) for offset in file.nextline().split()[-3:]])
            
            move_file_pointer_to_sibling(file)
            
        line = file.nextline().strip()